In [1]:
import argparse
import os
import sys
import json
import tensorflow as tf
from model import RecogModel

In [2]:
with open("./recog_params.json", 'r') as f:
    recog_params = json.load(f)

In [3]:
net = RecogModel(input_channel=1,
                klass=len(recog_params['labels']),
                batch_size=1,
                network_type=recog_params['network_type'],
                ctc_params=recog_params['ctc_params'],
                seq2seq_params=recog_params['seq2seq_params'])

In [4]:
input_image = tf.placeholder(tf.uint8)

In [5]:
# label_result = net.generate(input_image)

image, _, _, _ = net._preprocess(input_data=input_image,
                                  generate=True)
output = net._create_network(image)
_output = tf.transpose(output, perm=[1, 0, 2])
label_result = tf.nn.ctc_beam_search_decoder(inputs=_output,
                                             sequence_length=tf.expand_dims(tf.shape(output)[1], 0),
                                             beam_width=100)
# label_result = tf.nn.ctc_greedy_decoder(inputs=_output,
#                                              sequence_length=tf.expand_dims(tf.shape(output)[1], 0))
test = tf.argmax(input=output, dimension=2)

In [6]:
sess = tf.Session()

In [7]:
saver = tf.train.Saver()

In [8]:
saver.restore(sess, "model.ckpt-1140")

In [9]:
from scipy import misc
input_image_data = misc.imread('test.png', mode='L')

In [10]:
output_data, test_data, label_result_data = sess.run([output, test, label_result], feed_dict={input_image: input_image_data})

In [11]:
out_label = []
for l in label_result_data[0][0].values:
    if l < len(recog_params['labels']):
        out_label.append(recog_params['labels'][l])
    else:
        out_label.append(' ')
print "".join(out_label)

fx=10-12+12
